In [1]:
#https://github.com/ofajardo/pyreadr
#!pip install pyreadr
#!pip install -U pip setuptools
#!pip install feather-format

import random
import pyreadr
import os
import pandas as pd
import numpy as np 
import feather
import decimal

import warnings
warnings.filterwarnings('ignore')

'''
Script to generate simulation experiment results. 

'''

'\nScript to generate simulation experiment results. \n\n'

In [4]:
#download experimental data; 
#original source: Kosuke Imai; Michael Lingzhi Li, 2021, "Replication Data for: Experimental Evaluation of Individualized Treatment Rules", https://doi.org/10.7910/DVN/YCYTOB, Harvard Dataverse, V1

'''
* `Z`: A binary treatment (PSA provision)
* `D`: An ordinal decision (judge's release decision)
* `Y`: An outcome (FTA)
* Pre-treatment covariates: `Sex`, `White`, `SexWhite`, `Age`, `NCorNonViolentMisdemeanorCharge`, `ViolentMisdemeanorCharge`, `NonViolentFelonyCharge`, `ViolentFelonyCharge`, `PendingChargeAtTimeOfOffense`, `PriorMisdemeanorConviction`, `PriorFelonyConviction`, `PriorViolentConviction`, `PriorSentenceToIncarceration`, `PriorFTAInPastTwoYears`, `PriorFTAOlderThanTwoYears`, and `Staff_ReleaseRecommendation`.

See the table below for the details of the pre-treatment covariates:

| Variables | Description |
|:---------|:---------|
|`Sex`|male (1) or female (0)|
|`White`|white (1)or non-white (0)|
|`SexWhite`|the interaction between `Sex` and `White`|
|`Age`|age|
|`NCorNonViolentMisdemeanorCharge`| binary variable for current NC (parole violations) or non-violent misdemeanor charge|
|`ViolentMisdemeanorCharge`| binary variable for current violent misdemeanor charge|
|`NonViolentFelonyCharge`| binary variable for current non-violent felony charge|
|`ViolentFelonyCharge`| binary variable for current violent felony charge|
|`PendingChargeAtTimeOfOffense`|binary variable for pending charge (felony, misdemeanor, or both) at the time of offense|
|`PriorMisdemeanorConviction`|binary variable for prior conviction of misdemeanor|
|`PriorFelonyConviction`|binary variable for prior conviction of felony|
|`PriorViolentConviction`|four-level ordinal variable for prior violent conviction ($0,1,2$ and $3$, where $3$ indicates the counts of three or more)|
|`PriorSentenceToIncarceration`|binary variable for prior sentence to incarceration|
|`PriorFTAInPastTwoYears`|three-level ordinal variable for FTAs from past two years ($0,1$ and $2$ where $2$ indicates the counts of two or more)|
|`PriorFTAOlderThanTwoYears`|binary variable for FTAs from over two years ago|
|`Staff_ReleaseRecommendation`|four-level ordinal variable for the DMF recommendation|

'''
data_folder = "data_src/aihuman/data"
tot_res = list()

for filename in os.listdir(data_folder):
    f = data_folder+"/"+filename
    if ".rda" in f:
        result =pyreadr.read_r(f)
        tot_res.append(result) 

df_list = list()
ref_dict = dict()

for item in tot_res:
    for k, v in item.items():
        # print key and value
        ref_dict[k] = v



In [5]:
#turn data files into a single dataframe summarizing results

#ref_dict.keys() #index of each data file in data folder
ref_dict['NCAdata'] = ref_dict['NCAdata'].rename(columns={"Y": "Y_NCA"})
ref_dict['NVCAdata'] = ref_dict['NVCAdata'].rename(columns={"Y": "Y_NVCA"})
ref_dict['FTAdata'] = ref_dict['FTAdata'].rename(columns={"Y": "Y_FTA"})

res_df = pd.concat([ref_dict['PSAdata'],ref_dict['NCAdata']], axis = 1)
res_df["Y_NVCA"] = ref_dict['NVCAdata']["Y_NVCA"]
res_df["Y_FTA"] = ref_dict['FTAdata']["Y_FTA"]
res_df["Hearing Date"] = ref_dict['HearingDate']
res_df = res_df.loc[:,~res_df.columns.duplicated()].copy()
res_df.to_csv("exp_summary.csv")

#res_df["Hearing Date"].nunique() -- Note: Experimental results for 274 dates ; 1891 cases 

In [6]:
res_df_exp_data = pd.read_csv("exp_summary.csv") #-- access results for dataframe
res_df_exp_data

,Unnamed: 0,Z,D,FTAScore,NCAScore,NVCAFlag,DMF,Sex,White,SexWhite,...,PriorFelonyConviction,PriorViolentConviction,PriorSentenceToIncarceration,PriorFTAInPastTwoYears,PriorFTAOlderThanTwoYears,Staff_ReleaseRecommendation,Y_NCA,Y_NVCA,Y_FTA,Hearing Date
0,0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,2
1,1,1.0,1.0,4.0,4.0,0.0,0.0,1.0,0.0,0.0,...,1.0,2.0,1.0,1.0,1.0,3.0,0.0,0.0,1.0,2
2,2,1.0,2.0,2.0,3.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,6.0,0.0,0.0,0.0,4
3,3,1.0,0.0,5.0,4.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,1.0,1
4,4,0.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1886,1886,0.0,0.0,3.0,5.0,0.0,1.0,1.0,0.0,0.0,...,1.0,2.0,1.0,0.0,1.0,4.0,1.0,0.0,0.0,261
1887,1887,0.0,2.0,6.0,6.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,1.0,2.0,1.0,6.0,0.0,0.0,0.0,270
1888,1888,0.0,0.0,5.0,5.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,1.0,1.0,5.0,0.0,0.0,0.0,251
1889,1889,1.0,1.0,2.0,4.0,1.0,1.0,1.0,0.0,0.0,...,1.0,3.0,1.0,0.0,0.0,6.0,0.0,0.0,0.0,271


In [7]:
def run_trial(t, J_dist, res_df): 
    '''
    J_dist -- percentage of treated cases assigned to each judge; accomodates any variable number of judges
    res_df -- dataframe of experiment data with unassigned judge column 
    '''
    res_df = res_df.copy()
    #add an empty judge column 
    res_df["J"] = np.nan #judge assignment 
    res_df["T"] = np.nan #trial number 

    #re-set indices 
    num = len(res_df.index)
    div = len(J_dist.keys())
    k = [num // div + (1 if x < num % div else 0)  for x in range (div)]
    J_idx = dict()
    selected = list()

    curr = 0
    for j in J_dist.keys():
    
        p1 = J_dist[j]  
        p2 = 1.0 - J_dist[j] 

        pop1 = res_df.index[res_df['Z'] == 1.0].tolist()
        w1 = len(pop1)*[p1]
        pop2 = res_df.index[res_df['Z'] == 0.0].tolist()
        w2 = len(pop2)*[p2]

        pop = pop1+pop2
        w = w1+w2
    
        pop_dict = dict(zip(pop, w))
        pop_unselected = [x for x in pop if x not in selected]
    
        pop_dict = dict((i, pop_dict[i]) for i in pop_unselected if i in pop_dict)
        w = np.array(list(pop_dict.values()))
        w_final = w/w.sum()

        J_idx[j] = list(np.random.choice(list(pop_dict.keys()),k[curr],replace=False, p=w_final))
    
        curr+= 1
        
        selected += J_idx[j]
        for i in J_idx[j]:
            res_df.at[i, "J"]= j
            res_df.at[i, "T"]= t

    return res_df

In [8]:
def normalize_outcomes(df):
    '''
    Staff_ReleaseRecommendation -- four-level ordinal variable for the DMF recommendation
    DMF -- binary decision
    
    Any of the negative outcomes (NCA, NVCA, FTA) counts as a negative outcome overall. 
    Any decision beyond release (D = 0) is counted as a decision/ action towards detainment (D = 1). 
    A decision is correct when the action (eg. release, D = 0) matches the actual outcome (eg. Y = 0). 

    Get final decisions and outcomes into binary format. 
    
    '''
    df = df.copy()
    TP=0
    TN=0
    FP=0
    FN=0
    D_norm_list = list()
    Y_list = list()
    D_corr_def_list = list()

    for index, row in df.iterrows():
        if row["Y_NCA"]== 1.0 or row["Y_NVCA"]== 1.0 or row["Y_FTA"]== 1.0:
            Y=1.0
        else:
            Y=0.0
        
        if Y==1.0 and row["DMF"]== 1.0:
            TP+=1
        if Y==0.0 and row["DMF"]== 0.0:
            TN+=1
        if Y==1.0 and row["DMF"]== 0.0:
            FN+=1
        if Y==0.0 and row["DMF"]== 1.0:
            FP+=1

        if row["D"] > 0:
            D_norm = 1.0
        elif row["D"] == 0:
            D_norm = 0.0

        if D_norm == 1:
            D_corr = "U" #detained defendents cannot be reliably observed
        elif D_norm == Y:
            D_corr = 1.0 #if released and did not recidivate or commit a crime 
        else:
            D_corr = 0.0 #if released and did recidivate or commit a crime

        Y_list.append(Y)
        D_norm_list.append(D_norm)
        D_corr_def_list.append(D_corr)
        
    #set a normalized outcome 
    df["Y"] = Y_list
    df["D_norm"] = D_norm_list

    #identify default correct decisions  
    df["D_corr_def"] = D_corr_def_list

    return df 


def calc_acc(df, DMF_id):
    '''
    Calculate the accuracy of a given predictive recommendation with respect to downstream outcomes. 
    
    '''
    df = df.copy()
    TP=0
    TN=0
    FP=0
    FN=0
    D_norm_list = list()
    Y_list = list()
    D_corr_def_list = list()

    for index, row in df.iterrows():
        Y = row["Y"]
        if Y==1.0 and row[DMF_id]== 1.0:
            TP+=1
        if Y==0.0 and row[DMF_id]== 0.0:
            TN+=1
        if Y==1.0 and row[DMF_id]== 0.0:
            FN+=1
        if Y==0.0 and row[DMF_id]== 1.0:
            FP+=1

    accuracy = (TP + TN) / (TP + TN + FP + FN)
    return accuracy
    
#Original exp data results: Model accruacy -- (0.5367530407191962)

In [9]:
def f1_decision_sim(res_df, J_dist, b_k=0.1, thres=0.6, linear = True, exp_idx = "D_exp1"):
    '''
    Running simulation for treatment exposure model. 

    Note that the manipulatable experiment design choice is treatment assignment. 
    '''
    
    res_df = res_df.copy()
    exp = dict()
    for k in list(J_dist.keys()):
        if k:
            exp[k] = [0, 0]


    D_list = list()
    a = 1-b_k

    J_ik=0

    for index, row in res_df.iterrows():
    #going through all the cases

        #set default decision
        def_ik = row["D_norm"]
        rec_ik = row["DMF"]

        judge_name = row["J"]  # the judge for this case
        judge_cases_seen = exp[judge_name][1]
        judge_num_treated = exp[judge_name][0]

        if judge_cases_seen == 0:
            J_ik = b_k
        else:
            if linear:
                J_ik = b_k + a*(judge_num_treated/judge_cases_seen)  ## linear J_ik model 
            else:
                # avg exposure to PSA is above threshold
                if (judge_num_treated/judge_cases_seen)  > thres:
                    J_ik = 1 
                else:
                    #not above threshold 
                    J_ik = 0 #b_k + 0


        e_ik = np.random.binomial(1, p=J_ik)

        if row["Z"] == 0 :
            D_list.append(def_ik)
            exp[judge_name][1] += 1

        else:
            exp[judge_name][0] += 1
            exp[judge_name][1] += 1
            if e_ik == 1:
                D_list.append(rec_ik)
            elif e_ik == 0:
                D_list.append(def_ik)

    res_df[exp_idx] = D_list
    
    return res_df , exp_idx

In [10]:
def f2_decision_sim(res_df, J_dist, b_k=0.1, pred_thres=5, cap_thres=0.5, linear=True, exp_idx = "D_exp2"):
    '''
    Running simulation for capacity constraint model. 

    Note that the manipulatable experiment design choice is setting the model recommendation threshold 
    to control the model's positive predictive rate. 
    '''

    res_df = res_df.copy()

    #look at alternative recommendation with different positive predictions 
    res_df["DMF_alt_cc"] = np.where(res_df["Staff_ReleaseRecommendation"] > pred_thres, 1.0, 0.0) 
    #Note: higher threshold for less positive recommendation
    
    exp = dict()
    for k in list(J_dist.keys()):
        if k:
            exp[k] = [0, 0, 0]


    D_list = list()
    a = 1-b_k

    J_ik=0

    for index, row in res_df.iterrows():
    #going through all the cases

        #set default decision
        def_ik = row["D_norm"]
        rec_ik = row["DMF_alt_cc"]


        judge_name = row["J"]  # the judge for this case
        pos_num = exp[judge_name][2] #num positive cases each judge has seen 
        judge_cases_seen = exp[judge_name][1]
        judge_num_treated = exp[judge_name][0]
        
        if pos_num == 0:
            J_ik = b_k
        else:
            if linear:
                J_ik = b_k + a*(pos_num/judge_cases_seen)  ## linear J_ik
            else:
                # rate of exposure to positive cases is above threshold
                if (pos_num/judge_cases_seen)  > cap_thres:
                    J_ik = 1 
                else:
                    #not above thres
                    J_ik = 0 


        e_ik = np.random.binomial(1, p=J_ik)

        if row["Z"] == 0 :
            D_list.append(def_ik)
            exp[judge_name][1] += 1

        else:
            exp[judge_name][0] += 1
            exp[judge_name][1] += 1
            if rec_ik > 0:
                exp[judge_name][2] += 1
                
            if e_ik == 1:
                D_list.append(rec_ik)
            elif e_ik == 0:
                D_list.append(def_ik)

    res_df[exp_idx] = D_list
    
    return res_df , exp_idx

In [11]:
def generate_alt_DMF_acc(res_df, acc_boost=0.5):
    '''
    Generating alternative model recommendations, based on an accuracy boost from the default. 

    Note: acc_boost at 1 is going to give perfectly accurate recommendations; acc_boost at 0 won't change anything from default DMF model 

    Note that a decision is considered correct when the action (eg. release, D = 0) matches the actual outcome (eg. Y = 0), 
    so we boost accuracy by assigning the correct recommendation to a random subset of incorrect default decisions. 
    '''    
    
    res_df = res_df.copy()
    rdf = normalize_outcomes(res_df)
    DMF_acc = calc_acc(rdf, "DMF")

    DMF_alt_acc = list()
    for index, row in rdf.iterrows():
        flip_score = np.random.binomial(1, p=acc_boost)
        if row["DMF"] != row["Y"]:
            if flip_score:
                DMF_alt_acc.append(row["Y"])
            else:
                DMF_alt_acc.append(row["DMF"])
        else:
            DMF_alt_acc.append(row["DMF"])
 

    rdf["DMF_alt_acc"] = DMF_alt_acc
    DMF_alt_acc = calc_acc(rdf, "DMF_alt_acc")


    return rdf, DMF_acc, DMF_alt_acc

In [12]:
def f3_decision_sim(res_df, J_dist, b_k=0.1, acc_boost=0.5, thres=0.5, linear=True, exp_idx = "D_exp3"):
    '''
    Running simulation for low trust model. 

    Note that the manipulatable experiment design choice is model recommendation accuracy.
    '''    

    res_df = res_df.copy()
    res_df, DMF_acc, DMF_alt_acc = generate_alt_DMF_acc(res_df, acc_boost)
    
    exp = dict()
    for k in list(J_dist.keys()):
        if k:
            exp[k] = [0, 0, 0]


    D_list = list()
    a = 1-b_k

    J_ik=0


    for index, row in res_df.iterrows():
    #going through all the cases

        #set default decision
        def_ik = row["D_norm"]
        rec_ik = row["DMF_alt_acc"]


        judge_name = row["J"]  # the judge for this case
        incorr_num = exp[judge_name][2] #num incorrect predictions each judge has seen 
        judge_cases_seen = exp[judge_name][1]
        judge_num_treated = exp[judge_name][0]
        

        if incorr_num == 0:
            J_ik = b_k
        else:
            if linear:
                J_ik = b_k + a*(incorr_num/judge_cases_seen)  # linear 
            else:
                if (incorr_num/judge_cases_seen)  < thres:
                    #avg exposure to incorrect recommendation is above threshold
                    J_ik = 1 #non-linear
                else:
                    #not above thres
                    J_ik = 0 
            

        e_ik = np.random.binomial(1, p=J_ik)

        if rec_ik != row["Y"]:
            wrong = True
        else:
            wrong = False 

        if row["Z"] == 0 :
            D_list.append(def_ik)
            exp[judge_name][1] += 1

        else:
            exp[judge_name][0] += 1
            exp[judge_name][1] += 1
            if wrong:
                exp[judge_name][2] += 1
                
            if e_ik == 1:
                D_list.append(rec_ik)
            elif e_ik == 0:
                D_list.append(def_ik)

    res_df[exp_idx] = D_list
    
    
    return res_df, exp_idx

In [13]:
def calc_corr_decisions(df, D_label):
    '''
    Identify correct decisions. 

    Note that a decision is considered correct when the action (eg. release, D = 0) matches the actual outcome (eg. Y = 0). 
    '''
    df = df.copy() 
    D_corr_list = list()

    for index, row in df.iterrows():
        if row[D_label] == 1:
            D_corr = "U" #detained defendents cannot be reliably observed
        elif row["Y"] == row[D_label]:
            D_corr = 1.0 #if released and did not recidivate or commit a crime 
        else:
            D_corr = 0.0 #if released and did recidivate or commit a crime 

        D_corr_list.append(D_corr)

    #identify correct decisions  
    df["D_corr"] = D_corr_list

    return df 

In [16]:
##Experiment 1 

#Run 1000 trials of simulation for each scenario and bsave to a csv
#Demo: Run 10 trials 

results_folder = "Results/Experiment_1/"

n_trials = 10
J_scenarios = [{"J1": 0.6, "J2": 0.1, "J3": 0.3}, {"J1": 0.333, "J2": 0.333, "J3": 0.333}, {"J1": 0.5, "J2": 0.5}, {"J1": 0.999999999, "J2": 0.0000001}]

for J_dist in J_scenarios:
    all_res = list()
    scenario_name = str()
    
    for v in J_dist.values():
        scenario_name = scenario_name+"_"+str(v)
     
    
    for i in range (1, n_trials+1):
        t_res = run_trial(i, J_dist, res_df_exp_data)
        all_res.append(t_res)

    exp_df = pd.concat(all_res)
    exp_df = normalize_outcomes(exp_df)
    
    exp_df, exp_idx = f1_decision_sim(exp_df, J_dist, 0.1, 0.5, True) #linear 
    exp_df = calc_corr_decisions(exp_df, exp_idx)

    print(J_dist)
    exp_df.to_csv(results_folder+exp_idx+"_summary"+scenario_name+"_t"+str(n_trials)+".csv")

    exp_df, exp_idx = f1_decision_sim(exp_df, J_dist, 0.1, 0.5, False) #non-linear 
    exp_df = calc_corr_decisions(exp_df, exp_idx)
    
    exp_df.to_csv(results_folder+exp_idx+"_summary"+scenario_name+"_t"+str(n_trials)+"_non_linear.csv")

{'J1': 0.6, 'J2': 0.1, 'J3': 0.3}
{'J1': 0.333, 'J2': 0.333, 'J3': 0.333}
{'J1': 0.5, 'J2': 0.5}
{'J1': 0.999999999, 'J2': 1e-07}


In [19]:
## Experiment 2 

#Run 1000 trials of simulation for each treatment assignment scenario; and each threshold level then save to a csv
#Demo: Run 10 trials

results_folder = "Results/Experiment_2/"

n_trials = 10
J_scenarios = [{"J1": 0.6, "J2": 0.1, "J3": 0.3}, {"J1": 0.333, "J2": 0.333, "J3": 0.333}, {"J1": 0.5, "J2": 0.5}, {"J1": 0.999999999, "J2": 0.0000001}]

for J_dist in J_scenarios:
    all_res = list()
    scenario_name = str()
    
    for v in J_dist.values():
        scenario_name = scenario_name+"_"+str(v)
     
    
    for i in range (1, n_trials+1):
        t_res = run_trial(i, J_dist, res_df_exp_data)
        all_res.append(t_res)

    exp_df = pd.concat(all_res)
    exp_df = normalize_outcomes(exp_df)
    
    for i in range(3,7):
        print(i, J_dist)
        exp_df, exp_idx = f2_decision_sim(exp_df, J_dist, 0.1, i, 0.5, True, "D_exp2") #linear
        exp_df = calc_corr_decisions(exp_df, exp_idx)
        exp_df.to_csv(results_folder+exp_idx+"_summary"+scenario_name+"_t"+str(n_trials)+"_"+str(i)+".csv")
        
        exp_df, exp_idx = f2_decision_sim(exp_df, J_dist, 0.1, i, 0.5, False, "D_exp2") #linear
        exp_df = calc_corr_decisions(exp_df, exp_idx)
        exp_df.to_csv(results_folder+exp_idx+"_summary"+scenario_name+"_t"+str(n_trials)+"_"+str(i)+"_non_linear.csv")

3 {'J1': 0.6, 'J2': 0.1, 'J3': 0.3}
4 {'J1': 0.6, 'J2': 0.1, 'J3': 0.3}
5 {'J1': 0.6, 'J2': 0.1, 'J3': 0.3}
6 {'J1': 0.6, 'J2': 0.1, 'J3': 0.3}
3 {'J1': 0.333, 'J2': 0.333, 'J3': 0.333}
4 {'J1': 0.333, 'J2': 0.333, 'J3': 0.333}
5 {'J1': 0.333, 'J2': 0.333, 'J3': 0.333}
6 {'J1': 0.333, 'J2': 0.333, 'J3': 0.333}
3 {'J1': 0.5, 'J2': 0.5}
4 {'J1': 0.5, 'J2': 0.5}
5 {'J1': 0.5, 'J2': 0.5}
6 {'J1': 0.5, 'J2': 0.5}
3 {'J1': 0.999999999, 'J2': 1e-07}
4 {'J1': 0.999999999, 'J2': 1e-07}
5 {'J1': 0.999999999, 'J2': 1e-07}
6 {'J1': 0.999999999, 'J2': 1e-07}


In [18]:
## Experiment 3 

#Run 1000 trials of simulation for each scenario; and each accuracy boost level then save to a csv
#Demo: Run 10 trials

results_folder = "Results/Experiment_3/"

n_trials = 10
J_scenarios = [{"J1": 0.6, "J2": 0.1, "J3": 0.3}, {"J1": 0.333, "J2": 0.333, "J3": 0.333}, {"J1": 0.5, "J2": 0.5}, {"J1": 0.999999999, "J2": 0.0000001}]

for J_dist in J_scenarios:
    all_res = list()
    scenario_name = str()
    
    for v in J_dist.values():
        scenario_name = scenario_name+"_"+str(v)
     
    #assign treatment 
    for i in range (1, n_trials+1):
        t_res = run_trial(i, J_dist, res_df_exp_data)
        all_res.append(t_res)

    exp_df = pd.concat(all_res)
    exp_df = normalize_outcomes(exp_df)
    
    for i in np.arange(0, 1.1, 0.1):
        i = round(i,1)
        print(i, J_dist)
        exp_df, exp_idx = f3_decision_sim(exp_df, J_dist, 0.1, i, 0.5, True, "D_exp3") #linear
        exp_df = calc_corr_decisions(exp_df, exp_idx)
        exp_df.to_csv(results_folder+exp_idx+"_summary"+scenario_name+"_t"+str(n_trials)+"_"+str(i)+".csv")

        exp_df, exp_idx = f3_decision_sim(exp_df, J_dist, 0.1, i, 0.5, False, "D_exp3") #non-linear
        exp_df = calc_corr_decisions(exp_df, exp_idx)
        exp_df.to_csv(results_folder+exp_idx+"_summary"+scenario_name+"_t"+str(n_trials)+"_"+str(i)+"_non_linear.csv")
    

0.0 {'J1': 0.6, 'J2': 0.1, 'J3': 0.3}
0.1 {'J1': 0.6, 'J2': 0.1, 'J3': 0.3}
0.2 {'J1': 0.6, 'J2': 0.1, 'J3': 0.3}
0.3 {'J1': 0.6, 'J2': 0.1, 'J3': 0.3}
0.4 {'J1': 0.6, 'J2': 0.1, 'J3': 0.3}
0.5 {'J1': 0.6, 'J2': 0.1, 'J3': 0.3}
0.6 {'J1': 0.6, 'J2': 0.1, 'J3': 0.3}
0.7 {'J1': 0.6, 'J2': 0.1, 'J3': 0.3}
0.8 {'J1': 0.6, 'J2': 0.1, 'J3': 0.3}
0.9 {'J1': 0.6, 'J2': 0.1, 'J3': 0.3}
1.0 {'J1': 0.6, 'J2': 0.1, 'J3': 0.3}
0.0 {'J1': 0.333, 'J2': 0.333, 'J3': 0.333}
0.1 {'J1': 0.333, 'J2': 0.333, 'J3': 0.333}
0.2 {'J1': 0.333, 'J2': 0.333, 'J3': 0.333}
0.3 {'J1': 0.333, 'J2': 0.333, 'J3': 0.333}
0.4 {'J1': 0.333, 'J2': 0.333, 'J3': 0.333}
0.5 {'J1': 0.333, 'J2': 0.333, 'J3': 0.333}
0.6 {'J1': 0.333, 'J2': 0.333, 'J3': 0.333}
0.7 {'J1': 0.333, 'J2': 0.333, 'J3': 0.333}
0.8 {'J1': 0.333, 'J2': 0.333, 'J3': 0.333}
0.9 {'J1': 0.333, 'J2': 0.333, 'J3': 0.333}
1.0 {'J1': 0.333, 'J2': 0.333, 'J3': 0.333}
0.0 {'J1': 0.5, 'J2': 0.5}
0.1 {'J1': 0.5, 'J2': 0.5}
0.2 {'J1': 0.5, 'J2': 0.5}
0.3 {'J1': 0.5, '